In [ ]:
import math
import highspy

In [ ]:
import marimo as mo
import nbformat
import util

# MILP 定式化 (HiGHS)

数理最適化モデルを用いて SCSP を解く.
定式化については SCIP 版を参照.

In [ ]:
class Model:
    def __init__(self, instance: list[str]):
        max_len = sum(len(s) for s in instance)

        highs = highspy.Highs()

        seqs = [
            [
                highs.addIntegral(lb=0, ub=max_len - 1)
                for idx, _ in enumerate(s)
            ]
            for s in instance
        ]
        for seq in seqs:
            for idx, _ in enumerate(seq):
                if idx == 0:
                    continue
                highs.addConstr(seq[idx - 1] + 1 <= seq[idx])

        for idx1, (s1, seq1) in enumerate(zip(instance, seqs)):
            for idx2, (s2, seq2) in enumerate(zip(instance, seqs)):
                if idx1 >= idx2:
                    continue

                for cidx1, (c1, cvar1) in enumerate(zip(s1, seq1)):
                    for cidx2, (c2, cvar2) in enumerate(zip(s2, seq2)):
                        if c1 != c2:
                            lt = highs.addBinary()
                            gt = highs.addBinary()
                            highs.addConstr(lt + gt == 1)
                            big_m = max_len
                            highs.addConstr(cvar1 + 1 <= cvar2 + big_m * (1 - lt))
                            highs.addConstr(cvar1 + big_m * (1 - gt) >= cvar2 + 1)

        obj = highs.addVariable(lb=0, ub=max_len)
        for seq in seqs:
            highs.addConstr(obj >= seq[-1])
        highs.setObjective(obj=obj, sense=highspy.ObjSense.kMinimize)

        self.instance = instance
        self.highs = highs
        self.seqs = seqs

    def solve(self, time_limit: int | None = 60) -> "Model":
        if time_limit is not None:
            self.highs.setOptionValue("time_limit", time_limit)
        self.highs.solve()
        return self

    def to_solution(self) -> str | None:
        info = self.highs.getInfo()
        primal_status = info.primal_solution_status
        if primal_status != highspy.SolutionStatus.kSolutionStatusFeasible:
            return None

        highssolution = self.highs.getSolution()
        objval = int(self.highs.getObjectiveValue())
        sol_char_idx = 0
        solution = ""
        while sol_char_idx <= objval:
            found = False
            for idx, (s, seq) in enumerate(zip(self.instance, self.seqs)):
                for c_idx, cvar in enumerate(seq):
                    if math.isclose(
                        highssolution.col_value[cvar.index],
                        sol_char_idx
                    ):
                        solution += s[c_idx]
                        found = True
                        sol_char_idx += 1
                if found:
                    break
            if not found:
                sol_char_idx += 1

        return solution

In [ ]:
def solve(instance: list[str], time_limit: int | None = 60) -> str | None:
    return Model(instance).solve(time_limit).to_solution()

In [ ]:
instance_01 = util.parse("uniform_q26n004k015-025.txt")
solution_01 = solve(instance_01)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 7665 rows; 5139 cols; 20384 nonzeros; 5138 integer variables (5054 binary)
Coefficient ranges:
  Matrix [1e+00, 8e+01]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 8e+01]
  RHS    [1e+00, 8e+01]
Presolving model


5138 rows, 2612 cols, 15330 nonzeros  0s


4987 rows, 2612 cols, 14877 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   4987 rows
   2612 cols (2527 binary, 84 integer, 1 implied int., 0 continuous, 0 domain fixed)
   14877 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   24              inf                  inf        0      0      0         0     0.1s


...

      6885       0         0   0.00%   25              74                66.22%       76      0      0    281612    43.1s


      6885       0         0   0.00%   25              74                66.22%       76     50     23    281728    43.1s


      7826      94       301   2.78%   25              74                66.22%    33031    983   2279    330670    48.4s


      8048     126       377   2.78%   25              74                66.22%     1058    906   2757    347410    53.4s


      8447     158       511   2.78%   25              74                66.22%     1003    899   3649    380350    60.0s
      8447     158       511   2.78%   25              74                66.22%     1003    899   3649    380350    60.0s

Solving report
  Status            Time limit reached
  Primal bound      74
  Dual bound        25
  Gap               66.22% (tolerance: 0.01%)
  P-D integral      39.8901535198
  Solution status   feasible
                    74 (objective)
                    0 (bound viol.)
                    7.1054273576e-15 (int. viol.)
                    0 (row viol.)
  Timing            60.02 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 5
  Nodes             8447
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     380350 (total)
                    1365 (strong br.)
                    38340 (separation)
                    66052 (heuristics)


In [ ]:
_instance = instance_01
_solution = solution_01

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 75) ---
 Sol: tkgnkuhmpxnhtqgxzvxulcinycosovozppljigqfolnbxxecvsuaqpvzissgbxrddbcsvrvnngf
str1: tkgnkuhmpxnhtqgxzvx---i----s-----------------------------------------------
str2: ----------------------i---o--------ji-qfolnbxx-cvsu-qpv-iss-bx------------f
str3: -----u--------------lcinycosovozppl------------------p---------------------
str4: ----------------------i--------------g--------e-v--a---z---gb-rddbcsvrvnngf

solution is feasible: True


In [ ]:
instance_02 = util.parse("uniform_q26n008k015-025.txt")
solution_02 = solve(instance_02)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 38584 rows; 25782 cols; 102774 nonzeros; 25781 integer variables (25606 binary)
Coefficient ranges:
  Matrix [1e+00, 2e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+02]
  RHS    [1e+00, 2e+02]


Presolving model


25781 rows, 12979 cols, 77168 nonzeros  0s


25096 rows, 12979 cols, 75113 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   25096 rows
   12979 cols (12803 binary, 175 integer, 1 implied int., 0 continuous, 0 domain fixed)
   75113 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   24              inf                  inf        0      0      0         0     0.6s


 R       0       0         0   0.00%   24              147               83.67%        0      0      0      6906     2.5s


         0       0         0   0.00%   25              147               82.99%    22513   4366    185     34501    25.7s


       101      71        15   0.00%   25              147               82.99%    24390   4392    234     62139    60.1s
       101      71        15   0.00%   25              147               82.99%    24390   4392    234     62139    60.1s

Solving report
  Status            Time limit reached
  Primal bound      147
  Dual bound        25
  Gap               82.99% (tolerance: 0.01%)
  P-D integral      47.8074669481
  Solution status   feasible
                    147 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            60.05 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 2
  Nodes             101
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     62139 (total)
                    0 (strong br.)
                    6669 (separation)
                    39233 (heuristics)


In [ ]:
_instance = instance_02
_solution = solution_02

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 148) ---
 Sol: ulcinygevacosovozpplrxwtknbczfjxqkrdrlctodtmprpxwgnkuhmpxnhvxerzbrvigpledcvdpfzsmsbroqvbbhtcsvrvnngyfqgxzplrzxucpmqvgtdfuxisolnbxxcvsuqpvissbxfcdsbo
str1: -----------------------tk------------------------gnkuhmpxnh-------------------------------t----------qgxz----------v-----xis------------------------
str2: ---i-------o------------------j------------------------------------i-----------------q--------------f-----------------------olnbxxcvsuqpvissbxf-----
str3: ulciny----cosovozppl------------------------p-------------------------------------------------------------------------------------------------------
str4: ---i--geva------z--------------------------------g--------------br------d--

In [ ]:
instance_03 = util.parse("uniform_q26n016k015-025.txt")
solution_03 = solve(instance_03)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 140897 rows; 94040 cols; 375510 nonzeros; 94039 integer variables (93716 binary)
Coefficient ranges:
  Matrix [1e+00, 3e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 3e+02]
  RHS    [1e+00, 3e+02]


Presolving model


94039 rows, 47182 cols, 281794 nonzeros  0s


91182 rows, 47182 cols, 273223 nonzeros  1s


Objective function is integral with scale 1

Solving MIP model with:
   91182 rows
   47182 cols (46858 binary, 323 integer, 1 implied int., 0 continuous, 0 domain fixed)
   273223 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   25              inf                  inf        0      0      0         0     3.9s


         0       0         0   0.00%   25              inf                  inf        0      0      6     24836    27.1s


         0       0         0   0.00%   25              inf                  inf   288925  11617      7     44359    56.8s


         0       0         0   0.00%   25              inf                  inf    95173  14460      8     47116    60.1s

Solving report
  Status            Time limit reached
  Primal bound      inf
  Dual bound        25
  Gap               inf
  P-D integral      0
  Solution status   -
  Timing            60.07 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 0
  Nodes             0
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     47116 (total)
                    0 (strong br.)
                    22280 (separation)
                    0 (heuristics)


In [ ]:
_instance = instance_03
_solution = solution_03

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution not found ---


In [ ]:
instance_04 = util.parse("uniform_q05n010k010-010.txt")
solution_04 = solve(instance_04)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 10945 rows; 7331 cols; 29120 nonzeros; 7330 integer variables (7230 binary)


Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+02]
  RHS    [1e+00, 1e+02]
Presolving model


7330 rows, 3716 cols, 21890 nonzeros  0s


6964 rows, 3716 cols, 20792 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   6964 rows
   3716 cols (3615 binary, 100 integer, 1 implied int., 0 continuous, 0 domain fixed)
   20792 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   10              inf                  inf        0      0      0         0     0.3s


...

       909     750        64   0.17%   10.00010305     55                81.82%     8214   1357    783    132237    32.8s


 L    1007     783        85   0.20%   10.00010305     51                80.39%    12461   1477    883    137755    37.8s


      1270     864       129   0.20%   10.00010305     51                80.39%    13087   1541   1064    182778    45.1s


      1396     953       146   0.20%   10.00010305     51                80.39%     1832   1647   1261    208471    52.1s


      1406     953       150   0.20%   10.00010305     51                80.39%     1890   1677   1310    228090    60.1s
      1406     953       150   0.20%   10.00010305     51                80.39%     1890   1677   1310    228090    60.1s

Solving report
  Status            Time limit reached
  Primal bound      51
  Dual bound        11
  Gap               78.43% (tolerance: 0.01%)
  P-D integral      42.6610020537
  Solution status   feasible
                    51 (objective)
                    0 (bound viol.)
                    2.79776202206e-14 (int. viol.)
                    0 (row viol.)
  Timing            60.06 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 6
  Nodes             1406
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     228090 (total)
                    0 (strong br.)
                    6671 (separation)
                    154888 (heuristics)


In [ ]:
_instance = instance_04
_solution = solution_04

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad

--- Solution (of length 52) ---
  Sol: dcbacdebadbaaeaebadacbeeadbccdeaeeadcbcdeedacbdbecde
str01: dcb-c---------------c----dbcc-e---------------------
str02: --b--d---db--e-e------ee--b--d----------------------
str03: -c-acde------e------c-e---b---e---------------------
str04: ---a--e--d--------d------d---de------b-d--d---------
str05: ---ac--b-----e-e----c---a-bc--e---------------------
str06: --b----ba-b--e--b-d-cb--a---------------------------
str07: --b----ba----eaebada--------------------------------
str08: ------e------e-e------e----c---------b-d-----b--e--e
str09: -c--cde------e----da-----d-c-d----------------------
str10: --b--d--a-b-------d--be-a------a---d----------------

solution is feasible: True


In [ ]:
instance_05 = util.parse("uniform_q05n050k010-010.txt")
solution_05 = solve(instance_05)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 294593 rows; 196563 cols; 785248 nonzeros; 196562 integer variables (196062 binary)
Coefficient ranges:
  Matrix [1e+00, 5e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 5e+02]
  RHS    [1e+00, 5e+02]


Presolving model


196562 rows, 98532 cols, 589186 nonzeros  0s


186760 rows, 98532 cols, 559780 nonzeros  2s


Objective function is integral with scale 1

Solving MIP model with:
   186760 rows
   98532 cols (98031 binary, 500 integer, 1 implied int., 0 continuous, 0 domain fixed)
   559780 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   10              inf                  inf        0      0      0         0     4.3s


         0       0         0   0.00%   10              inf                  inf        0      0      1     22207    60.2s

Solving report
  Status            Time limit reached
  Primal bound      inf
  Dual bound        10
  Gap               inf
  P-D integral      0
  Solution status   -
  Timing            60.15 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 0
  Nodes             0
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     22207 (total)
                    0 (strong br.)
                    0 (separation)
                    0 (heuristics)


In [ ]:
_instance = instance_05
_solution = solution_05

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 5 chars) ---
str01: dcbccdbcce
str02: bddbeeeebd
str03: cacdeecebe
str04: aeddddebdd
str05: acbeecabce
str06: bbabebdcba
str07: bbaeaebada
str08: eeeecbdbee
str09: ccdeedadcd
str10: bdabdbeaad
str11: ededaaaeaa
str12: aaeaabeeac
str13: eaabcaccdb
str14: bdeeadeade
str15: caedadeeed
str16: ebcadbabbe
str17: ddceeabdea
str18: dabcddeaec
str19: aadceedaab
str20: aeecceeeaa
str21: bbdaecaade
str22: dacedaedab
str23: aaeabbbbce
str24: dedbcbcaab
str25: dbdaaebbcb
str26: debedbebac
str27: ceebcdcbde
str28: dbedaadaab
str29: cccdcbebdc
str30: aeeacdbcbd
str31: dacbeacccd
str32: ecebccdbdb
str33: ddbbcedabb
str34: aaeabaaeba
str35: ecbbcaadcd
str36: debccecdbc
str37: daacbaeebc
str38: adabeaacce
str39: daecdbacaa
str40: dacbbdcedc
str41: dedbeebbde
str42: cdadcdcdaa
str43: ceedcbaeed
str44: ceaecaaaca
str45: dcccebbbad
str46: baeeaebbde
str47: dbdebaccdb
str48: ebcbeedaea
str49: aeeebbdbca
str50: dbdabcecbb

--- Solution not found ---


In [ ]:
instance_06 = util.parse("nucleotide_n010k010.txt")
solution_06 = solve(instance_06)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms
MIP  has 9256 rows; 6205 cols; 24616 nonzeros; 6204 integer variables (6104 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+02]
  RHS    [1e+00, 1e+02]
Presolving model


6204 rows, 3153 cols, 18512 nonzeros  0s


5981 rows, 3153 cols, 17843 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   5981 rows
   3153 cols (3052 binary, 100 integer, 1 implied int., 0 continuous, 0 domain fixed)
   17843 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   10              inf                  inf        0      0      0         0     0.1s


 R       0       0         0   0.00%   10              30                66.67%        0      0      0      1800     0.2s


...

      2606      25        65   0.00%   11              26                57.69%    16797   1277    466    303002    41.0s


 L    2671      44        85   0.20%   11              25                56.00%     1394   1291    606    304470    47.4s


 L    2780      29       117   0.20%   11              23                52.17%    16895   1317    835    334691    51.6s


      2887      51       143   0.21%   11              23                52.17%     2128   1368    964    384318    58.2s


      2928      49       158   0.21%   11              23                52.17%     3753    382   1051    391615    60.0s

Solving report
  Status            Time limit reached
  Primal bound      23
  Dual bound        11
  Gap               52.17% (tolerance: 0.01%)
  P-D integral      34.8248826328
  Solution status   feasible
                    23 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            60.02 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 4
  Nodes             2928
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     391615 (total)
                    0 (strong br.)
                    28330 (separation)
                    124994 (heuristics)


In [ ]:
_instance = instance_06
_solution = solution_06

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: CTTTTGATCT
str02: CTGCATGCTT
str03: TTGTAGATCT
str04: ATAACTAATT
str05: AGGTTTATAC
str06: CTTTTGATCT
str07: TTGTAGATCT
str08: TTGTAGATCT
str09: TTGTAGATCT
str10: TAATATTACC

--- Solution (of length 24) ---
  Sol: CTTAGCTATGATTACCGTATATCT
str01: CTT---T-TGAT--C--T------
str02: CT--GC-ATG----C--T-T----
str03: -TT-G-TA-GAT--C--T------
str04: ---A--TA--A---C--TA-AT-T
str05: ---AG----G-TT----TATA-C-
str06: CTT---T-TGAT--C--T------
str07: -TT-G-TA-GAT--C--T------
str08: -TT-G-TA-GAT--C--T------
str09: -TT-G-TA-GAT--C--T------
str10: -T-A---AT-ATTACC--------

solution is feasible: True


In [ ]:
instance_07 = util.parse("nucleotide_n050k050.txt")
solution_07 = solve(instance_07)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 6732070 rows; 4488881 cols; 17950520 nonzeros; 4488880 integer variables (4486380 binary)


Coefficient ranges:
  Matrix [1e+00, 2e+03]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+03]
  RHS    [1e+00, 2e+03]


Presolving model


4488880 rows, 2245691 cols, 13464140 nonzeros  11s


4448576 rows, 2245691 cols, 13343228 nonzeros  60s


Presolve: Time limit reached

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0    61.2s

Solving report
  Status            Time limit reached
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  P-D integral      0
  Solution status   -
  Timing            61.18 (total)
                    0.00 (presolve)
    

In [ ]:
_instance = instance_07
_solution = solution_07

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 4 chars) ---
str01: CTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGT
str02: CTGCATGCTTAGTGCACTCACGCAGTATAATTAATAACTAATTACTGTCG
str03: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str04: ATAACTAATTACTGTCGTTGACAGGACACGAGTAACTCGTCTATCTTCTG
str05: AGGTTTATACCTTCCTAGGTAACAAACCAACCAACTTTCGATCTCTTGTA
str06: CTTTTGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGT
str07: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str08: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str09: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str10: TAATATTACCTGATGGCCGCGCCCCTCAAAAAGTGGGCCCTTGGACAGAT
str11: TAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTT
str12: ACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAA
str13: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str14: ACCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTTTAA
str15: GTTAACAATAATCACACCATCACCGTTTTTTCAAGCGGGAAAAAATAGCC
str16: TTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC
str17: AGATCTGTTCTCTAAACGAACTTTAAAATCTG

In [ ]:
instance_08 = util.parse("protein_n010k010.txt")
solution_08 = solve(instance_08)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 11872 rows; 7949 cols; 31592 nonzeros; 7948 integer variables (7848 binary)
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 1e+02]
  RHS    [1e+00, 1e+02]
Presolving model


7948 rows, 4025 cols, 23744 nonzeros  0s


7732 rows, 4025 cols, 23096 nonzeros  0s
Objective function is integral with scale 1

Solving MIP model with:
   7732 rows
   4025 cols (3924 binary, 100 integer, 1 implied int., 0 continuous, 0 domain fixed)
   23096 nonzeros



Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   10              inf                  inf        0      0      0         0     0.1s


...

      1963     138       599  57.14%   11              16                31.25%    20841   1095   4049    300415    42.8s


      2336     182       712  57.94%   11              16                31.25%     8499   1628   4754    350323    47.8s


      2719     230       824  59.50%   11              16                31.25%     4123   1273   5522    401771    52.8s


      3090     241       946  59.81%   11              16                31.25%    17390   1185   6238    449061    57.9s


      3213     257       987  59.82%   11              16                31.25%    10190   1209   6439    468724    60.2s

Solving report
  Status            Time limit reached
  Primal bound      16
  Dual bound        11
  Gap               31.25% (tolerance: 0.01%)
  P-D integral      20.1297174418
  Solution status   feasible
                    16 (objective)
                    0 (bound viol.)
                    0 (int. viol.)
                    0 (row viol.)
  Timing            60.23 (total)
                    0.00 (presolve)
                    0.00 (solve)
                    0.00 (postsolve)
  Max sub-MIP depth 2
  Nodes             3213
  Repair LPs        0 (0 feasible; 0 iterations)
  LP iterations     468724 (total)
                    0 (strong br.)
                    22442 (separation)
                    28630 (heuristics)


In [ ]:
_instance = instance_08
_solution = solution_08

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 9 chars) ---
str01: MESLVPGFNE
str02: MESLVPGFNE
str03: MFVFLVLLPL
str04: MESLVPGFNE
str05: MESLVPGFNE
str06: MESLVPGFNE
str07: MFVFLVLLPL
str08: MFVFLVLLPL
str09: MESLVPGFNE
str10: MFVFLVLLPL

--- Solution (of length 17) ---
  Sol: MEFSLVPGFLVLNLPEL
str01: ME-SLVPGF---N--E-
str02: ME-SLVPGF---N--E-
str03: M-F--V--FLVL-LP-L
str04: ME-SLVPGF---N--E-
str05: ME-SLVPGF---N--E-
str06: ME-SLVPGF---N--E-
str07: M-F--V--FLVL-LP-L
str08: M-F--V--FLVL-LP-L
str09: ME-SLVPGF---N--E-
str10: M-F--V--FLVL-LP-L

solution is feasible: True


In [ ]:
instance_09 = util.parse("protein_n050k050.txt")
solution_09 = solve(instance_09)

Running HiGHS 1.11.0 (git hash: 364c83a): Copyright (c) 2025 HiGHS under MIT licence terms


MIP  has 8489785 rows; 5660691 cols; 22637760 nonzeros; 5660690 integer variables (5658190 binary)


Coefficient ranges:
  Matrix [1e+00, 2e+03]
  Cost   [1e+00, 1e+00]
  Bound  [1e+00, 2e+03]
  RHS    [1e+00, 2e+03]


Presolving model


5660690 rows, 2831596 cols, 16979570 nonzeros  14s


5626342 rows, 2831596 cols, 16876526 nonzeros  73s


Presolve: Time limit reached

Src: B => Branching; C => Central rounding; F => Feasibility pump; J => Feasibility jump;
     H => Heuristic; L => Sub-MIP; P => Empty MIP; R => Randomized rounding; Z => ZI Round;
     I => Shifting; S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0    75.1s

Solving report
  Status            Time limit reached
  Primal bound      inf
  Dual bound        -inf
  Gap               inf
  P-D integral      0
  Solution status   -
  Timing            75.14 (total)
                    0.00 (presolve)
    

In [ ]:
_instance = instance_09
_solution = solution_09

util.show(_instance)
if _solution is not None:
    util.show(_instance, _solution)
    print(f"solution is feasible: {util.is_feasible(_instance, _solution)}")
else:
    print("--- Solution not found ---")

--- Condition (with 20 chars) ---
str01: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str02: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str03: MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str04: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str05: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str06: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str07: MFVFLVLLPLVSSQCVNLITRTQSYTNSFTRGVYYPDKVFRSSVLHSTKD
str08: MFVFLVLLPLVSSQCVNLRTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str09: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str10: MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str11: MFVFLVLLPLVSSQCVMPLFNLITTTQSYTNFTRGVYYPDKVFRSSVLHL
str12: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str13: MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str14: MDPIINGSSANVYLTDSYLKGVISFSECNALGSYLFNGPYLKNDYTNLIS
str15: MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHS
str16: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGFGDSVEEVLSEARQHLKDGT
str17: MESLVPGFNEKTHVQLSLPVLQVRDVLVRGF